# RL asmt 1

Discuss in detail how Markov Decision Process (MDP) can help in route planning application.
Justify your answer in 350-400 words. Explain in detail your environment, state space, action
space and equations aligned with the given problem. Write the answer in the Colab Cell itself.
